<a href="https://colab.research.google.com/github/reallygooday/60daysofudacity/blob/master/Index_Error_Generating_Text_with_Character_Level_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Text with a Character-Level RNN


https://colab.research.google.com/drive/1IdUnFCBl24SpVVdZnvUsgfmTbwf1vp6v#scrollTo=5AVUH0AHuN-T

https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb

In [30]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [31]:
!pip install unidecode

In [0]:
import unidecode
import string
import random
import re


In [33]:
!ls '/content/drive/My Drive/Text Generator'

'generated_script2019-07-30 17:15:13.735175.txt'
 hemingway_stories.gdoc
 hemingway_stories.txt
 Obama_Inaugural-Address-20-January-2009.gdoc
 oliver.gdoc
 oliver.txt
 place2.txt
 place.txt
'preprocessed2019-07-30 17:06:00.732826.p'
'preprocessed2019-07-30 17:06:42.389736.p'
'preprocessed2019-07-30 17:14:12.698497.p'
'preprocessed2019-07-30 17:17:30.349953.p'
 sel_hn_stories.csv
 sel_hn_stories.gsheet
'trained_scriptGenModel2019-07-30 17:15:09.497845.pt'
'trained_scriptGenModel2019-07-30 17:16:10.999422.pt'
'trained_scriptGenModel2019-07-30 17:16:37.995327.pt'
'trained_scriptGenModel2019-07-30 17:17:48.526540.pt'
'trained_scriptGenModel2019-07-30 17:17:50.015998.pt'


In [34]:
all_characters = string.printable
n_characters = len(all_characters)

file = unidecode.unidecode(open('/content/drive/My Drive//Text Generator/oliver.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 917048


In [35]:
chunk_len = 200

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

know what you're
doing?'

'Know what I'm--Oh!' cried Sikes, turning to Fagin, 'she's out of her
senses, you know, or she daren't talk to me in that way.'

'You'll drive me on the something desperate,' 


In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [37]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(char_tensor('abcDEF'))


tensor([10, 11, 12, 39, 40, 41])


In [0]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [0]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [0]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [0]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / chunk_len

In [43]:
n_epochs = 2000
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(*random_training_set())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Wh', 100), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

IndexError: ignored

In [0]:
print(evaluate('Th', 200, temperature=0.8))